___
<a href='https://cafe.naver.com/jmhonglab'><p style="text-align:center;"><img src='https://lh3.googleusercontent.com/lY3ySXooSmwsq5r-mRi7uiypbo0Vez6pmNoQxMFhl9fmZJkRHu5lO2vo7se_0YOzgmDyJif9fi4_z0o3ZFdwd8NVSWG6Ea80uWaf3pOHpR4GHGDV7kaFeuHR3yAjIJjDgfXMxsvw=w2400'  class="center" width="50%" height="50%"/></p></a>
___
<center><em>Content Copyright by HongLab, Inc.</em></center>

# [AsyncIO](https://docs.python.org/3/library/asyncio.html)

Asynchronous IO라는 일반적인 개념을 파이썬 패키지로 구현  
이름에서 볼 수 있듯이 동시성으로 여러 개의 비동기 IO-Bound 작업(task)들을 관리


### Blocking vs Non-Blocking

Blocking은 직렬(Serial)로 실행할 수 밖에 없음  
사운드 카드에게 출력을 요청하는 경우: 대기할 수도 있고 바로 다른 출력을 요청할 수도 있다.

In [2]:
# Blocking
import simpleaudio as sa

# p1 시작
p1 = sa.WaveObject.from_wave_file("sound_files/25.wav").play()
p1.wait_done() # p1이 끝날 때까지 반환하지 않음

# p1이 끝나야 p2가 시작
p2 = sa.WaveObject.from_wave_file("sound_files/29.wav").play()
p2.wait_done()

# p2가 끝나야 p3가 시작
p3 = sa.WaveObject.from_wave_file("sound_files/32.wav").play()
p3.wait_done()

In [3]:
# Non-Blocking
import simpleaudio as sa

p1 = sa.WaveObject.from_wave_file("sound_files/25.wav").play()
p2 = sa.WaveObject.from_wave_file("sound_files/29.wav").play()
p3 = sa.WaveObject.from_wave_file("sound_files/32.wav").play()

# p1.wait_done()
# p2.wait_done()
# p3.wait_done()

### AsyncIO의 기본적인 사용방법

##### ```async```, ```await```

In [4]:
import asyncio
import time

async def my_task():
    print("Hello")
    await asyncio.sleep(1) # Non-blocking sleep
    #time.sleep(1) # Blocking sleep
    print("World")
    return "OK"

# print(type(my_task())) # <class 'coroutine'>

async def main():
    t1 = asyncio.create_task(my_task())
    t2 = asyncio.create_task(my_task()) 
    # print(type(t1)) # <class '_asyncio.Task'>

    result = await asyncio.gather(t1, t2)

    print(result)

if __name__ == "__main__":
    # asyncio.run(main()) # 스크립트 모드
    await main() # 주피터 노트북에서 실행시킬 경우

Hello
World
Hello
World
['OK', 'OK']


Python 3.4 이하 제너레이터 코루틴 옛날 방식

In [26]:
# DeprecationWarning: "@coroutine" decorator is deprecated since Python 3.8, use "async def" instead def count():
@asyncio.coroutine
def my_task():
    print("Hello")
    yield from asyncio.sleep(1) # Asyc sleep
    print("World")
    return "OK"

async def main():
    result = await asyncio.gather(my_task(), my_task())
    print(result)

if __name__ == "__main__":
    # asyncio.run(main()) # 스크립트 모드
    await main() # 주피터 노트북에서 실행시킬 경우

C:\Users\jmhong\AppData\Local\Temp\ipykernel_6596\3820692385.py:4: DeprecationWarning: "@coroutine" decorator is deprecated since Python 3.8, use "async def" instead
  def my_task():


Hello
Hello
World
World
['OK', 'OK']


##### 쓰레딩과 같이 사용하기

In [5]:
import asyncio
import time
import threading

lock = threading.Lock()

def blocked_task(task_name):
    with lock:
        print(task_name, str(threading.current_thread().name))
        print(task_name, "Hello")

    time.sleep(1) # Blocking sleep

    with lock:
        print(task_name, "World")
    
    return "OK"

async def main():

    loop = asyncio.get_running_loop()

    f1 = loop.run_in_executor(None, blocked_task, "Task1")
    f2 = loop.run_in_executor(None, blocked_task, "Task2")

    #print(type(f1)) # <class '_asyncio.Future'>

    result = await asyncio.gather(f1, f2)

    print(result)

if __name__ == "__main__":
    # asyncio.run(main()) # 스크립트 모드
    await main() # 주피터 노트북에서 실행시킬 경우

Task1 asyncio_0
Task1 Hello
Task2 asyncio_1
Task2 Hello
Task2 World
Task1 World
['OK', 'OK']


##### [실습] 화음 재생

스텝1. blocking, non-blocking 둘 다 해보기

In [7]:
import simpleaudio as sa
import asyncio
import time

async def play_wav(filename, block = False):
    p1 = sa.WaveObject.from_wave_file(filename).play()
    if block:
        p1.wait_done() # p1이 끝날 때까지 반환하지 않음

async def main():
    pass

if __name__ == "__main__":
    # asyncio.run(main()) # 스크립트 모드
    await main() # 주피터 노트북에서 실행시킬 경우

스텝2. ```run_in_executor()```를 사용해서 blocking을 non-blocking으로 만들어서 화음 재생하기

In [ ]:
import simpleaudio as sa
import asyncio
import time

def play_wav(filename, block = True):
    p1 = sa.WaveObject.from_wave_file(filename).play()
    if block:
        p1.wait_done() # p1이 끝날 때까지 반환하지 않음

async def main():
    pass

if __name__ == "__main__":
    # asyncio.run(main()) # 스크립트 모드
    await main() # 주피터 노트북에서 실행시킬 경우

##### [실습] AsyncIO로 온도가 가장 높은 도시 찾기

.py 파일 사용